### **Loading Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import keras
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, MaxPool2D, Flatten, Dense, Input, Reshape, Dropout, InputLayer
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from keras.models import Sequential, Model
from keras.utils import to_categorical
import pandas as pd
import numpy as np
import cv2
import os
from keras.optimizers import Adam
from keras.losses import MeanSquaredError, SparseCategoricalCrossentropy
from keras.metrics import SparseCategoricalAccuracy
np.random.seed(11)

In [ ]:
train_dir = os.path.join(os. getcwd() , r'/content/drive/MyDrive/train')
test_dir = os.path.join (os.getcwd(), r'/content/drive/MyDrive/test')
categories = ['benign','malignant']
y_train = []
x_train = []
x_test = []
y_test = []
Batch_Size = 32
INIT_LR = 1e-4
EPOCHES = 10

In [ ]:
for category in categories:
  path = os.path.join(train_dir, category)
  for img in os.listdir(path):
    img_path = os.path.join(path, img)
    image = cv2.imread(img_path)
    image = cv2.resize(image, (224,224))
    image = image / 255
    x_train.append(image)
    y_train.append(category)

In [ ]:
for category in categories:
  path = os.path.join(test_dir, category)
  for img in os.listdir(path):
    img_path = os.path.join(path, img)
    image = cv2.imread(img_path)
    image - cv2.resize(image, (224,224))
    image = image / 255
    x_test.append(image)
    y_test.append(category)

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from keras.utils import to_categorical

lb = LabelBinarizer ()
y_train = lb.fit_transform(y_train)
y_train = to_categorical(y_train,num_classes=2)
y_test = lb.fit_transform(y_test)
y_test = to_categorical (y_test, num_classes=2)

In [ ]:
x_train = np.array(x_train, dtype= 'float32')
y_train = np.array(y_train,dtype= 'float32')
x_test = np.array(x_test,dtype= 'float32')
y_test = np.array(y_test, dtype= 'float32')

In [ ]:
x_train.shape

(2637, 224, 224, 3)

In [ ]:
# x_train = x_train [:1100]
# x_test = x_test [:300]
# y_train = y_train [:1100]
# y_test = y_test [:300]

In [ ]:
x_test.shape

(660, 224, 224, 3)

In [ ]:
# x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:]))) #Flattn the 224*224*3 images into vectors of size 150528
# x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

x_train = x_train.reshape(-1, 150528)  # Reshape to (number_of_samples, 150528)
x_test = x_test.reshape(-1, 150528)  # Reshape to (number_of_samples, 150528)

In [ ]:
x_train.shape

(2637, 150528)

### **Building Autoncoder "1"**

In [ ]:
input_dim = 150528
input_shape = (input_dim,)

# Encoder
encoder_input = Input(shape=input_shape)
encoded = Dense(128, activation='relu') (encoder_input)
encoded = Dense(64, activation= 'relu') (encoded)

# Decoder
decoded = Dense(64, activation= 'relu') (encoded)
decoded = Dense(128, activation='relu') (decoded)
decoded = Dense(input_dim, activation='sigmoid') (decoded) # Last Decoder Output

In [ ]:
# Classification Layer
classification_output = Dense(2, activation='softmax') (decoded)

autoencoder = Model(encoder_input, [decoded, classification_output])

In [ ]:
# Compile the autoencoder model
autoencoder.compile(optimizer=Adam(learning_rate=INIT_LR),
                    loss=['mean_squared_error', 'categorical_crossentropy'],
                    loss_weights=[0.5, 0.5],  # You can adjust this weight
                    metrics=['accuracy'])

In [ ]:
autoencoder.fit(x_train, [x_train, y_train],
                epochs=EPOCHES,
                batch_size=Batch_Size,
                shuffle=True,
                verbose = 2)

Epoch 1/10
83/83 - 60s - loss: 0.6212 - dense_4_loss: 0.0507 - dense_5_loss: 1.1918 - dense_4_accuracy: 0.0000e+00 - dense_5_accuracy: 0.5199 - 60s/epoch - 721ms/step
Epoch 2/10
83/83 - 57s - loss: 0.3679 - dense_4_loss: 0.0507 - dense_5_loss: 0.6851 - dense_4_accuracy: 0.0000e+00 - dense_5_accuracy: 0.6265 - 57s/epoch - 690ms/step
Epoch 3/10
83/83 - 60s - loss: 0.3336 - dense_4_loss: 0.0508 - dense_5_loss: 0.6164 - dense_4_accuracy: 0.0000e+00 - dense_5_accuracy: 0.7091 - 60s/epoch - 718ms/step
Epoch 4/10
83/83 - 60s - loss: 0.3853 - dense_4_loss: 0.0507 - dense_5_loss: 0.7199 - dense_4_accuracy: 0.0000e+00 - dense_5_accuracy: 0.7042 - 60s/epoch - 724ms/step
Epoch 5/10
83/83 - 59s - loss: 0.3446 - dense_4_loss: 0.0510 - dense_5_loss: 0.6382 - dense_4_accuracy: 0.0000e+00 - dense_5_accuracy: 0.7213 - 59s/epoch - 713ms/step
Epoch 6/10
83/83 - 59s - loss: 0.3021 - dense_4_loss: 0.0509 - dense_5_loss: 0.5534 - dense_4_accuracy: 0.0000e+00 - dense_5_accuracy: 0.7505 - 59s/epoch - 707ms/ste

### **Building Autoencoder"2"**

In [ ]:
input_dim = 150528
input_shape = (input_dim,)

# Encoder
encoder_input = Input(shape=input_shape)
encoded = Dense(128, activation='relu') (encoder_input)
encoded = Dense(64, activation= 'relu') (encoded)

# Decoder
decoded = Dense(64, activation= 'relu') (encoded)
decoded = Dense(128, activation='relu') (decoded)
decoded = Dense(input_dim, activation='sigmoid') (decoded) # Last Decoder Output

In [ ]:
# Classification Layer
classification_output = Dense(2, activation='softmax') (encoded)

autoencoder = Model(encoder_input, [decoded, classification_output])

In [ ]:
# Compile the autoencoder model
autoencoder.compile(optimizer=Adam(learning_rate=INIT_LR),
                    loss=['mean_squared_error', 'categorical_crossentropy'],
                    loss_weights=[0.5, 0.5],  # You can adjust this weight
                    metrics=['accuracy'])

In [ ]:
autoencoder.fit(x_train, [x_train, y_train],
                epochs=EPOCHES,
                batch_size=Batch_Size,
                shuffle=True,
                verbose = 2)

Epoch 1/10
83/83 - 52s - loss: 0.4208 - dense_10_loss: 0.0279 - dense_11_loss: 0.8137 - dense_10_accuracy: 3.7922e-04 - dense_11_accuracy: 0.6579 - 52s/epoch - 630ms/step
Epoch 2/10
83/83 - 51s - loss: 0.3730 - dense_10_loss: 0.0208 - dense_11_loss: 0.7251 - dense_10_accuracy: 3.7922e-04 - dense_11_accuracy: 0.6796 - 51s/epoch - 616ms/step
Epoch 3/10
83/83 - 52s - loss: 0.3274 - dense_10_loss: 0.0208 - dense_11_loss: 0.6340 - dense_10_accuracy: 3.7922e-04 - dense_11_accuracy: 0.7179 - 52s/epoch - 632ms/step
Epoch 4/10
83/83 - 49s - loss: 0.3264 - dense_10_loss: 0.0207 - dense_11_loss: 0.6321 - dense_10_accuracy: 0.0000e+00 - dense_11_accuracy: 0.7228 - 49s/epoch - 592ms/step
Epoch 5/10
83/83 - 50s - loss: 0.2958 - dense_10_loss: 0.0207 - dense_11_loss: 0.5708 - dense_10_accuracy: 7.5844e-04 - dense_11_accuracy: 0.7304 - 50s/epoch - 599ms/step
Epoch 6/10
83/83 - 55s - loss: 0.2623 - dense_10_loss: 0.0206 - dense_11_loss: 0.5040 - dense_10_accuracy: 3.7922e-04 - dense_11_accuracy: 0.7649

### **Building Autoencoder"5"**

In [ ]:
model = Sequential ()
model.add(InputLayer (input_shape= (224, 224, 3)))
model.add(Conv2D (32, kernel_size= (2,2),activation='relu'))
model.add (MaxPooling2D (pool_size=(2,2)))
model.add(Conv2D (64, kernel_size= (2,2),activation='relu'))
model.add(MaxPooling2D (pool_size= (2,2)))
model.add(Conv2D (128,kernel_size= (2,2),activation='relu'))
model.add(MaxPooling2D(pool_size= (2,2)))
model.add(Flatten ())

model.add(Dense (128, activation='relu'))
model.add (Dropout (0.5))
model.add (Dense (2,activation= 'softmax' ))

In [ ]:
#Regularization
optimzation = Adam(learning_rate=INIT_LR, weight_decay=INIT_LR/Batch_Size)

In [ ]:
model.compile (optimizer=optimzation, loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
x_train = Reshape((224, 224, 3))(x_train)

In [ ]:
History = model.fit(x_train, y_train,
                    epochs=EPOCHES,
                    batch_size=Batch_Size)

Epoch 1/10
79/83 [===========================>..] - ETA: 0s

ValueError: ignored